In [1]:
import os
%pwd

'c:\\Users\\aarav\\Downloads\\IRIS-MLops\\research'

In [2]:
os.chdir('../')

In [3]:
os.environ['MLFLOW_TRACKING_URI']='https://dagshub.com/harsh-priyam/IRIS-MLops.mlflow'
os.environ['MLFLOW_TRACKING_USERNAME']='harsh-priyam'
os.environ['MLFLOW_TRACKING_PASSWORD']='d34676be0f7f95c5afd000a5be7be52ce1ba8d12'

In [4]:
# Entity

In [49]:
from dataclasses import dataclass
from pathlib import Path 
 
@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path 
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [50]:
from IRIS_MLops.constants import *
from IRIS_MLops.utils.common import read_yaml, create_directories,save_json

In [51]:

class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH):
        
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
            self.schema = read_yaml(schema_filepath)

            create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
          config = self.config.model_evaluation 
          params = self.params.LogisticRegression
          schema = self.schema.TARGET_COLUMN

          create_directories([config.root_dir])

          model_evaluation_config = ModelEvaluationConfig(
                root_dir=config.root_dir,
                test_data_path=config.test_data_path,
                model_path= config.model_path,
                all_params=params,
                metric_file_name=config.metric_file_name,
                target_column= schema.name,
                mlflow_uri="https://dagshub.com/harsh-priyam/IRIS-MLops.mlflow"
          )

          return model_evaluation_config

In [52]:
#Component

In [53]:
import os
import pandas as pd
from sklearn.metrics import accuracy_score,precision_score,recall_score
from urllib.parse import urlparse
import mlflow 
import mlflow.sklearn 
import numpy as np 
import joblib

In [54]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config 

    def eval_metrics(self,actual,pred):
        accuracy = accuracy_score(actual,pred)
        # precision = precision_score(actual,pred)
        # recall = recall_score(actual,pred)
        return accuracy
    
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)


        model = joblib.load(self.config.model_path)
        
        test_x = test_data.drop([self.config.target_column],axis=1)
        test_y = test_data[[self.config.target_column]]

       
        


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)

            (accuracy) = self.eval_metrics(test_y, predicted_qualities)

            # Saving metrics as local 

            scores = {"accuracy": accuracy}
            save_json(path=Path(self.config.metric_file_name),data = scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("accuracy", accuracy)
            # mlflow.log_metric("precision",precision)
            # mlflow.log_metric("recall", recall) 


            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # Please refer to the doc for more info
                mlflow.sklearn.log_model(model, "model",registered_model_name="LogisticRegression")
            else:
                mlflow.sklearn.log_model(model,"model")

In [55]:
# Pipeline

In [56]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2024-01-13 19:28:33,973: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-13 19:28:33,984: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-13 19:28:33,988: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-01-13 19:28:33,991: INFO: common: created directory at: artifacts]
[2024-01-13 19:28:33,993: INFO: common: created directory at: artifacts/model_evaluation]
[2024-01-13 19:28:34,566: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


c:\Users\aarav\Downloads\IRIS-MLops\irisenv\lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
c:\Users\aarav\Downloads\IRIS-MLops\irisenv\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'LogisticRegression' already exists. Creating a new version of this model...
2024/01/13 19:28:43 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: LogisticRegression, version 3
Created version '3' of 